In [47]:
import kotlinx.serialization.json.Json
import java.io.File
import io.github.lexadiky.jsonquery.*

val json = Json.parseToJsonElement(
    File("dataset.json").readText()
)

In [48]:
// Using path
println(json.query { path("shelter.name") })

"Purrfect Paws Cat Shelter"


In [49]:
// Using path with index
println(json.query { path("shelter.cats")[0].path("name") })
println(json.query { path("shelter.cats.0.name") })

"Mittens"
"Mittens"


In [50]:
// Using path with in array lookup
println(json.query { path("shelter.cats.name") })

["Mittens","Shadow","Mumpkin"]


In [51]:
// Using path with filter
println(json.query { path("shelter.cats.name").filterT<String> { it.startsWith("M") } })

["Mittens","Mumpkin"]


In [52]:
// Slices
println(json.query { path("shelter.cats")[0..1].path("name") })

["Mittens","Shadow"]


In [53]:
// Statistical operations
println(json.query { path("shelter.cats.age").max() })
println(json.query { path("shelter.cats.age").min() })
println(json.query { path("shelter.cats.age").average() })
println(json.query { path("shelter.cats.age").sum() })
println(json.query { path("shelter.cats.age").first() })
println(json.query { path("shelter.cats.age").last() })
println(json.query { path("shelter.cats.age").filterT<Int> { it >= 2 }.size() })

3.0
1.0
2.0
6.0
3
1
2


In [54]:
// Selecting specific fields
println(json.query { path("shelter.cats.0").select("name", "age") })
println(json.query { path("shelter.cats").select("name", "age") })

{"name":"Mittens","age":3}
[{"name":"Mittens","age":3},{"name":"Shadow","age":2},{"name":"Mumpkin","age":1}]


In [55]:
import kotlinx.serialization.json.buildJsonObject

// Aggregating data into object
println(json.query {
    path("shelter.cats").qmap {
        buildJsonObject {
            put("names", query { path("name") })
            put("ages", query { path("age") })
        }
    }
})

{"names":["Mittens","Shadow","Mumpkin"],"ages":[3,2,1]}
